# Wrangle comma-separated files with intercalated headers and varied column counts-with pandas and HURDAT2
In which we tackle "csv" files with intercalated header rows with pandas.

## Set up

You'll need quilt3 and pandas to run this notebook.
```sh
. activate YOUR_ENV
pip install quilt3[pyarrow] pandas
cd YOUR_CLEAN_DIR

```

In [4]:
import pandas as pd
import quilt3 as q3

## Get the data (be sure you're in a clean dir)

In [5]:
! mkdir ../data
! quilt3 install examples/hurdat2 --registry s3://quilt-example --dest ../data

Loading manifest: 100%|█████████████████████████████| 7/7 [00:00<00:00, 20.6k/s]
Copying objects: 100%|██████████████████████| 7.30M/7.30M [00:00<00:00, 461MB/s]


## Find dummy separator
We need a character that's NOT in the file, `%` works.
```sh
grep "%" external/hurdat2-1851-2020-052921.txt
```


## Ingest as single-column of strings

In [6]:
df = pd.read_csv("../data/external/hurdat2-1851-2020-052921.txt", sep="%", header=None)
df

,0
0,"AL011851, UNNAMED, 14,"
1,"18510625, 0000, , HU, 28.0N, 94.8W, 80, -99..."
2,"18510625, 0600, , HU, 28.0N, 95.4W, 80, -99..."
3,"18510625, 1200, , HU, 28.0N, 96.0W, 80, -99..."
4,"18510625, 1800, , HU, 28.1N, 96.5W, 80, -99..."
...,...
54636,"20201117, 1200, , HU, 13.7N, 84.7W, 75, 96..."
54637,"20201117, 1800, , TS, 13.7N, 85.7W, 55, 98..."
54638,"20201118, 0000, , TS, 13.8N, 86.7W, 40, 100..."
54639,"20201118, 0600, , TS, 13.8N, 87.8W, 35, 100..."


## Check structure
How many different types of rows are we dealing with?

In [7]:
widths = set()

for s in df[0]:
    widths.add(len(s.split(',')))
    
widths

{4, 21}

## Organize into a dict of arrays
Each header row maps to an array of its child data rows.

> *Warning* user arrays not dataframes as intermediates otherwise it's crazy slow.

In [8]:
storms = {}
header = None
for s in df[0]:
    tokens = s.split(',')
    assert (len(tokens) == 4) or len(tokens) == 21, 'unexpected row width'
    if len(tokens) == 4:
        header = s
        assert not (header in storms), f'repeated header: {header}'
        storms[header] = []
    else:
        assert header, 'no header before these data rows'
        row = [t.strip() for t in tokens]
        storms[header].append(row)

## Integrate header rows into child data rows
See the PDF in `examples/HURDAT2` for details on how these data are formatted.

In [9]:
frames = []
for k in storms:
    code, name, entries, _ = [t.strip() for t in k.split(',')]
    columns = {
        'Basin': code[:2],
        'ATCF Cyclone Number': code[2:4],
        'HYear': code[4:9],
        'Name': name,
        'Num. best track entries': entries
        
    }
    table = pd.DataFrame(storms[k], columns=[
        'YYYY-MM-DD',
        'TimeUTC',
        'Record identifier',
        'Status of system',
        'Latitude',
        'Longitude',
        'Max. sustained wind (knots)',
        'Min. pressure (millibars)',
        '34 kt wind max. NE (nautical miles)',
        '34 kt wind max. SE (nautical miles)',
        '34 kt wind max. SW (nautical miles)',
        '34 kt wind max. NW (nautical miles)',
        '50 kt wind max. NE (nautical miles)',
        '50 kt wind max. SE (nautical miles)',
        '50 kt wind max. SW (nautical miles)',
        '50 kt wind max. NW (nautical miles)',
        '64 kt wind max. NE (nautical miles)',
        '64 kt wind max. SE (nautical miles)',
        '64 kt wind max. SW (nautical miles)',
        '64 kt wind max. NW (nautical miles)',
        'DROPME'
    ])
    # add header columns
    for c in columns:
        table[c] = columns[c]
    table
    
    frames.append(table)
# concat arrays of arrays in ONE shot, making ONE dataframe = muuuch faster
master = pd.concat(frames)
master
   

,YYYY-MM-DD,TimeUTC,Record identifier,Status of system,Latitude,Longitude,Max. sustained wind (knots),Min. pressure (millibars),34 kt wind max. NE (nautical miles),34 kt wind max. SE (nautical miles),...,64 kt wind max. NE (nautical miles),64 kt wind max. SE (nautical miles),64 kt wind max. SW (nautical miles),64 kt wind max. NW (nautical miles),DROPME,Basin,ATCF Cyclone Number,HYear,Name,Num. best track entries
0,18510625,0000,,HU,28.0N,94.8W,80,-999,-999,-999,...,-999,-999,-999,-999,,AL,01,1851,UNNAMED,14
1,18510625,0600,,HU,28.0N,95.4W,80,-999,-999,-999,...,-999,-999,-999,-999,,AL,01,1851,UNNAMED,14
2,18510625,1200,,HU,28.0N,96.0W,80,-999,-999,-999,...,-999,-999,-999,-999,,AL,01,1851,UNNAMED,14
3,18510625,1800,,HU,28.1N,96.5W,80,-999,-999,-999,...,-999,-999,-999,-999,,AL,01,1851,UNNAMED,14
4,18510625,2100,L,HU,28.2N,96.8W,80,-999,-999,-999,...,-999,-999,-999,-999,,AL,01,1851,UNNAMED,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,20201117,1200,,HU,13.7N,84.7W,75,965,170,110,...,30,0,0,20,,AL,31,2020,IOTA,26
22,20201117,1800,,TS,13.7N,85.7W,55,988,150,70,...,0,0,0,0,,AL,31,2020,IOTA,26
23,20201118,0000,,TS,13.8N,86.7W,40,1000,140,0,...,0,0,0,0,,AL,31,2020,IOTA,26
24,20201118,0600,,TS,13.8N,87.8W,35,1005,140,0,...,0,0,0,0,,AL,31,2020,IOTA,26


In [10]:
# make sure we aren't losing anything
master['DROPME'].unique()

array([''], dtype=object)

In [11]:
master = master.drop(columns=['DROPME'])

In [12]:
master[master['Name'] == 'ALPHA']

,YYYY-MM-DD,TimeUTC,Record identifier,Status of system,Latitude,Longitude,Max. sustained wind (knots),Min. pressure (millibars),34 kt wind max. NE (nautical miles),34 kt wind max. SE (nautical miles),...,50 kt wind max. NW (nautical miles),64 kt wind max. NE (nautical miles),64 kt wind max. SE (nautical miles),64 kt wind max. SW (nautical miles),64 kt wind max. NW (nautical miles),Basin,ATCF Cyclone Number,HYear,Name,Num. best track entries
0,19720523,1800,,SD,31.2N,78.2W,25,1004,-999,-999,...,-999,-999,-999,-999,-999,AL,01,1972,ALPHA,24
1,19720524,0000,,SD,31.7N,77.0W,30,-999,-999,-999,...,-999,-999,-999,-999,-999,AL,01,1972,ALPHA,24
2,19720524,0600,,SD,32.2N,76.6W,30,1003,-999,-999,...,-999,-999,-999,-999,-999,AL,01,1972,ALPHA,24
3,19720524,1200,,SD,32.9N,76.2W,30,-999,-999,-999,...,-999,-999,-999,-999,-999,AL,01,1972,ALPHA,24
4,19720524,1800,,SD,33.5N,75.7W,30,-999,-999,-999,...,-999,-999,-999,-999,-999,AL,01,1972,ALPHA,24
5,19720525,0000,,SD,34.0N,75.4W,30,1004,-999,-999,...,-999,-999,-999,-999,-999,AL,01,1972,ALPHA,24
6,19720525,0600,,SD,34.3N,75.1W,30,-999,-999,-999,...,-999,-999,-999,-999,-999,AL,01,1972,ALPHA,24
7,19720525,1200,,SD,34.4N,74.6W,30,-999,-999,-999,...,-999,-999,-999,-999,-999,AL,01,1972,ALPHA,24
8,19720525,1800,,SD,34.2N,74.1W,30,-999,-999,-999,...,-999,-999,-999,-999,-999,AL,01,1972,ALPHA,24
9,19720526,0000,,SS,34.0N,73.5W,45,-999,-999,-999,...,-999,-999,-999,-999,-999,AL,01,1972,ALPHA,24


In [13]:
hurs2020 = master[
    (master['HYear'] == '2020')
    & (master['Status of system'] == 'HU')
]
hurs2020

,YYYY-MM-DD,TimeUTC,Record identifier,Status of system,Latitude,Longitude,Max. sustained wind (knots),Min. pressure (millibars),34 kt wind max. NE (nautical miles),34 kt wind max. SE (nautical miles),...,50 kt wind max. NW (nautical miles),64 kt wind max. NE (nautical miles),64 kt wind max. SE (nautical miles),64 kt wind max. SW (nautical miles),64 kt wind max. NW (nautical miles),Basin,ATCF Cyclone Number,HYear,Name,Num. best track entries
10,20200725,1200,,HU,27.1N,96.0W,70,981,90,100,...,40,20,0,0,20,AL,08,2020,HANNA,18
11,20200725,1800,,HU,26.9N,96.8W,80,973,100,100,...,50,25,20,20,25,AL,08,2020,HANNA,18
12,20200725,2200,L,HU,26.8N,97.3W,80,973,100,100,...,50,25,20,20,25,AL,08,2020,HANNA,18
13,20200725,2315,L,HU,26.7N,97.5W,80,974,100,100,...,50,25,20,20,25,AL,08,2020,HANNA,18
14,20200726,0000,,HU,26.7N,97.6W,75,976,100,100,...,30,25,20,0,0,AL,08,2020,HANNA,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,20201116,1800,,HU,13.5N,82.3W,130,918,200,130,...,80,40,30,25,40,AL,31,2020,IOTA,26
18,20201117,0000,,HU,13.6N,83.0W,130,918,200,130,...,80,40,30,25,40,AL,31,2020,IOTA,26
19,20201117,0340,L,HU,13.6N,83.5W,125,922,200,130,...,70,40,30,20,35,AL,31,2020,IOTA,26
20,20201117,0600,,HU,13.7N,83.8W,110,935,190,130,...,60,35,25,15,25,AL,31,2020,IOTA,26


In [14]:
hurs2020['Name'].unique()

array(['HANNA', 'ISAIAS', 'LAURA', 'MARCO', 'NANA', 'PAULETTE', 'SALLY',
       'TEDDY', 'GAMMA', 'DELTA', 'EPSILON', 'ZETA', 'ETA', 'IOTA'],
      dtype=object)

## Save as Parquet (smaller, faster) so no one has to do this again :P

In [15]:
master.to_parquet("../data/Atlantic-HURDAT2.parquet")

## Wrap it into a data package
We can now take our clean data and wrap it into versioned, documented, immutable package for other to use. Now colleagues can build upon our work with a simple:
`quilt3 install examples/hurdat2 --registry s3://quilt-example --dest ../data`

See here for more:
https://open.quiltdata.com/b/quilt-example/packages/examples/hurdat2/